In [ ]:
# Create the Spark Session
import pyspark
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("Streaming from Kafka") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) # Even if we manually stop de job, it continues processing
    .config('spark.jars', f'/opt/spark/jars/spark-sql-kafka-0-10_2.12-{pyspark.__version__}.jar')
    .config("spark.sql.shuffle.partitions", 4)
    .master("spark://spark-master:7077") 
    .getOrCreate()
)

spark

In [ ]:
# Create the kafka_df to read from kafka

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "device-data")
    .option("startingOffsets", "earliest")
    .load()
)

kafka_df.printSchema()

In [ ]:
from utils import flatten_df

flattened_df = flatten_df(kafka_df)

In [ ]:
flattened_df.printSchema()

In [ ]:
# Write the output to console sink to check the output

(flattened_df
 .writeStream
 .format("console")
 .outputMode("append")
 #.option("checkpointLocation", f"/home/jovyan/streaming_checkpoint_dir/{spark.sparkContext.appName.replace(' ', '_')}")
 .start()
 .awaitTermination())

In [ ]:
spark.stop()